In [67]:
import geopandas as gpd
import pandas as pd
from json import load
from requests import Session
from os.path import exists
from re import sub

## Preparação: Requisição dos dados da API do Observatório

> Nota: Não é possível solicitar dados de vítima não fatal mais vítima fatal, pois retorna o código `http 500 - Internal Server Error` - Erro no tempo de execução. Será feito duas requisições separadas e inseridas em arquivos diferentes, e então feito a colagem em seguida.

In [68]:
session = Session()

params_nao_fatal = {"data":"[[\"Todos\"],[\"PORTO VELHO\"],[\"VÍTIMA NÃO FATAL\"],[\"Todos\"],[\"1/2019\"],[\"12/2020\"]]",
          "labels": "[\"Natureza_do_Acidente\",\"Municipio\",\"Consequencia\",\"Via_1\",\"Data_Inicial\",\"Data_Final\"]"}

params_fatal = {"data":"[[\"Todos\"],[\"PORTO VELHO\"],[\"VÍTIMA FATAL\"],[\"Todos\"],[\"1/2019\"],[\"12/2020\"]]",
          "labels": "[\"Natureza_do_Acidente\",\"Municipio\",\"Consequencia\",\"Via_1\",\"Data_Inicial\",\"Data_Final\"]"}

cookie = {'chave':'lgpd'}

if not exists('AcidentesdeTransitoNaoFatal.json'):
    req = Session.post(session, url='http://observatorio.sepog.ro.gov.br/TransitoPerfil/GetGeoData', json=params_nao_fatal, cookies=cookie, stream=True)
    with open('AcidentesdeTransitoNaoFatal.json', 'wb') as arq_acidentes:
        arq_acidentes.write(req.content)

if not exists('AcidentesdeTransitoFatal.json'):
    req = Session.post(session, url='http://observatorio.sepog.ro.gov.br/TransitoPerfil/GetGeoData',json=params_fatal,cookies=cookie, stream=True)
    with open('AcidentesdeTransitoFatal.json', 'wb') as arq_acidentes:
        arq_acidentes.write(req.content)

## Primeiro passo: Normalização do JSON
- Será normalizado usando um método do pandas: [json_normalize](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html)

Formato padrão do dado:
```
{
    "<tipo de acidente>": {
        "<id>": {
            "LATITUDE": <float>,
            "LONGITUDE": <float>,
            "CONSEQUENCIA": <string>,
            "MUNICIPIO": <string>,
            "VEICULO_1": <string>,
            "VEICULO_2": <string>,
            "DATA_DO_FATO": <datetime>,
            "FROTA": <int>
        }
    }
}
```

In [ ]:
with open('AcidentesdeTransitoNaoFatal.json', 'r') as arq_acidentes:
    acidentes = load(arq_acidentes)

    df = pd.DataFrame() # Crio um DataFrame vazio onde irá reunir os dados normalizados

    for tipo_acidente, acidente in acidentes.items(): # Para cada tipo de acidente ...
        for id_, informacoes in acidente.items(): # Para cada item do tipo de acidente...
            informacoes.update({'ID':id_})
            df_items = pd.json_normalize(informacoes)
            df = pd.concat((df, df_items), ignore_index=True)
df

In [ ]:
with open('AcidentesdeTransitoFatal.json', 'r') as arq_acidentes:
    acidentes = load(arq_acidentes)

    # Mesma lógica para este DF
    for tipo_acidente, acidente in acidentes.items():
        for id_, informacoes in acidente.items():
            informacoes.update({'ID':id_})
            df_items = pd.json_normalize(informacoes)
            df = pd.concat((df, df_items), ignore_index=True) # Como o DF já existe, utilizo ele para a inserção.
df

## Segundo passo: Limpeza dos dados
- Retirar números após a virgula + a virgula do ID
- Excluir dados sem LATLON
- Tratando dados LATLON para gerar Geometria válida
- Substituir valores vazios das colunas VEICULO_1 e VEICULO_2 com `NÃO INFORMADO`

#### Removendo a virgula mais números após

In [ ]:
# Se tiver algum id com virgula mais qualquer outra coisa depois
if df['ID'].str.contains(r'\d,.*?').astype(bool).any():
    dados_incorretos = df['ID'].loc[df['ID'].str.contains(r'\d,.*?')]
    # Para cada id, selec. da vírgula para frente e apague
    dados_corretos = dados_incorretos.apply(lambda id_: sub(r'(?=,).+','', id_))
    # Inserir dados corrigidos no DataFrame
    df.loc[dados_corretos.index, ['ID']] = dados_corretos.to_frame()['ID']

#### Remover dados que estão sem geo

In [ ]:
df.drop(df.loc[(df.LATITUDE == "SEM GEO") | (df.LONGITUDE == "SEM GEO")].index, inplace=True)
df

#### Tratando dados para gerar Geometria válida

##### 1. Procurar dados com caracteres especiais

In [ ]:
# Previnir que o LATLON tenha qualquer caracter especial OU 2 pontos OU 2 hífen
Regex_Sem_Carac_Espec = r'[\!\"\#\$%\&\'\(\)\*\+\,\/\:\;\<\=\>\?\@\[\]\^\_\`\{\|\}\~]+|[\.]{2,}|[\-]{2,}'

##### 2. Corrigir esses dados utilizando regex

In [ ]:
# Se tiver caracteres ou duplicações de caract. ...
if df['LATITUDE'].str.contains(Regex_Sem_Carac_Espec).astype(bool).any():
    # Pegar linhas que estão com problemas
    dados_incorretos = df['LATITUDE'].loc[df['LATITUDE'].str.contains(Regex_Sem_Carac_Espec)]
    # Consertar erros
    dados_corretos = dados_incorretos.apply(lambda lat: sub(r'[^\-\d\.]|(\.)(?=\.)|(\-)(?=\-)','', lat))
    # Inserir dados corrigidos no DataFrame
    df.loc[dados_corretos.index, ['LATITUDE']] = dados_corretos.to_frame()['LATITUDE']

if df['LONGITUDE'].str.contains(Regex_Sem_Carac_Espec).astype(bool).any():
    dados_incorretos = df['LONGITUDE'].loc[df['LONGITUDE'].str.contains(Regex_Sem_Carac_Espec)]
    dados_corretos = dados_incorretos.apply(lambda lat: sub(r'[^\-\d\.]|(\.)(?=\.)|(\-)(?=\-)','', lat))
    df.loc[dados_corretos.index, ['LONGITUDE']] = dados_corretos.to_frame()['LONGITUDE']

#### Substituir dados vazios com `NÃO INFORMADO`

In [ ]:
# Será utilizado o NÃO INFORMADO pois já existe no dado original, irei manter o padrão

# Selecione as tuplas da coluna VEICULO_1 que tenham a condição verdadeira e substitua com 'NÃO INFOMADO'
df.loc[df.VEICULO_1 == '', 'VEICULO_1'] = 'NÃO INFORMADO'
df.loc[df.VEICULO_2 == '', 'VEICULO_2'] = 'NÃO INFORMADO'

### Terceiro Passo: Converção de dados e criar GeoDataFrame
- Mudar tipo de dado da coluna DATA_DO_FATO para `datetime`
- Mudar tipo de dado das colunas LATITUDE E LONGITUDE para `float`
- Mudar tipo de dado da coluna ID para `int`
- Gerar um `GeoDataFrame` válido a partir do `DataFrame`

#### Convertendo coluna DATA_DO_FATO para datetime

In [ ]:
df.loc[:,'DATA_DO_FATO'] = pd.to_datetime(df['DATA_DO_FATO'])

#### Convertendo colunas LATITUDE e LONGITUDE para float

In [ ]:
df.loc[:, 'LATITUDE'] = pd.to_numeric(df['LATITUDE'], downcast='float')
df.loc[:, 'LONGITUDE'] = pd.to_numeric(df['LONGITUDE'], downcast='float')

#### Convertendo coluna ID para int

In [ ]:
df.loc[:,'ID'] = pd.to_numeric(df['ID'], downcast='signed')

#### Verificar se as conversões estão corretas

In [ ]:
df.dtypes

#### Gerar `GeoDataFrame` utilizando o construtor do GeoPandas

In [ ]:
# Gerar um GeoDataFrame a partir do DataFrame utilizando as colunas LATLON
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.LONGITUDE, df.LATITUDE), crs='EPSG:4674')
# Não é mais necessários as colunas...
gdf.drop(['LATITUDE', 'LONGITUDE'], axis=1, inplace=True)

gdf.head(20)